# Lab | Hypothesis Testing

**Objective**

Welcome to the Hypothesis Testing Lab, where we embark on an enlightening journey through the realm of statistical decision-making! In this laboratory, we delve into various scenarios, applying the powerful tools of hypothesis testing to scrutinize and interpret data.

From testing the mean of a single sample (One Sample T-Test), to investigating differences between independent groups (Two Sample T-Test), and exploring relationships within dependent samples (Paired Sample T-Test), our exploration knows no bounds. Furthermore, we'll venture into the realm of Analysis of Variance (ANOVA), unraveling the complexities of comparing means across multiple groups.

So, grab your statistical tools, prepare your hypotheses, and let's embark on this fascinating journey of exploration and discovery in the world of hypothesis testing!

**Challenge 1**

In this challenge, we will be working with pokemon data. The data can be found here:

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv

In [9]:
#libraries
import pandas as pd
import scipy.stats as st
import numpy as np



In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv")
df

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


- We posit that Pokemons of type Dragon have, on average, more HP stats than Grass. Choose the propper test and, with 5% significance, comment your findings.

In [14]:
# Load the dataset


# Filter the data for Dragon and Grass types
dragon_hp = df[df['Type 1'] == 'Dragon']['HP']
grass_hp = df[df['Type 1'] == 'Grass']['HP']

# Perform a two-sample t-test (one-tailed)
t_stat, p_value = ttest_ind(dragon_hp, grass_hp, alternative='greater', nan_policy='omit')

# Significance level
alpha = 0.05

# Interpretation
if p_value < alpha:
    result = "We reject the null hypothesis. Dragon-type Pokémon have significantly higher HP than Grass-type Pokémon at the 5% significance level."
else:
    result = "We fail to reject the null hypothesis. There is no significant evidence that Dragon-type Pokémon have higher HP than Grass-type Pokémon at the 5% significance level."

result


'We reject the null hypothesis. Dragon-type Pokémon have significantly higher HP than Grass-type Pokémon at the 5% significance level.'

- We posit that Legendary Pokemons have different stats (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed) when comparing with Non-Legendary. Choose the propper test and, with 5% significance, comment your findings.


In [15]:
# Separate Legendary and Non-Legendary Pokémon
legendary = df[df['Legendary'] == True]
non_legendary = df[df['Legendary'] == False]

# List of stats to compare
stats = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

# Perform t-tests and store results
results = []
alpha = 0.05  # Significance level

for stat in stats:
    t_stat, p_value = ttest_ind(legendary[stat], non_legendary[stat], nan_policy='omit')
    significant = p_value < alpha
    results.append({
        'Stat': stat,
        't-statistic': t_stat,
        'p-value': p_value,
        'Significant (alpha=0.05)': significant
    })

# Convert results to a DataFrame and display
results_df = pd.DataFrame(results)
display(results_df)



,Stat,t-statistic,p-value,Significant (alpha=0.05)
0,HP,8.036124,3.330648e-15,True
1,Attack,10.397321,7.827253e-24,True
2,Defense,7.181240,1.584223e-12,True
3,Sp. Atk,14.191406,6.314916e-41,True
4,Sp. Def,11.037751,1.843981e-26,True
5,Speed,9.765234,2.354075e-21,True


**Challenge 2**

In this challenge, we will be working with california-housing data. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


**We posit that houses close to either a school or a hospital are more expensive.**

- School coordinates (-118, 37)
- Hospital coordinates (-122, 34)

We consider a house (neighborhood) to be close to a school or hospital if the distance is lower than 0.50.

Hint:
- Write a function to calculate euclidean distance from each house (neighborhood) to the school and to the hospital.
- Divide your dataset into houses close and far from either a hospital or school.
- Choose the propper test and, with 5% significance, comment your findings.
 

In [12]:
# Load the dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv"
housing_data = pd.read_csv(url)

# School and Hospital coordinates
school_coords = (-118, 37)
hospital_coords = (-122, 34)

# Function to calculate Euclidean distance
def euclidean_distance(lat, lon, target_coords):
    target_lat, target_lon = target_coords
    return np.sqrt((lat - target_lat) ** 2 + (lon - target_lon) ** 2)

# Calculate distances to school and hospital
housing_data['dist_to_school'] = housing_data.apply(
    lambda row: euclidean_distance(row['longitude'], row['latitude'], school_coords), axis=1)
housing_data['dist_to_hospital'] = housing_data.apply(
    lambda row: euclidean_distance(row['longitude'], row['latitude'], hospital_coords), axis=1)

# Define closeness to school or hospital
housing_data['close_to_school_or_hospital'] = (housing_data['dist_to_school'] < 0.50) | (housing_data['dist_to_hospital'] < 0.50)

# Divide dataset into houses close and far from school or hospital
close_houses = housing_data[housing_data['close_to_school_or_hospital']]
far_houses = housing_data[~housing_data['close_to_school_or_hospital']]

# Perform t-test for house prices
t_stat, p_value = ttest_ind(close_houses['median_house_value'], far_houses['median_house_value'], nan_policy='omit')

# Significance level
alpha = 0.05

# Interpretation
if p_value < alpha:
    result = "We reject the null hypothesis. Houses close to either a school or hospital are significantly more expensive at the 5% significance level."
else:
    result = "We fail to reject the null hypothesis. There is no significant evidence that houses close to either a school or hospital are more expensive at the 5% significance level."

result


'We reject the null hypothesis. Houses close to either a school or hospital are significantly more expensive at the 5% significance level.'